In [2]:
import torch
torch.hub.list('pytorch/vision:v0.5.0')

Using cache found in C:\Users\sanika/.cache\torch\hub\pytorch_vision_v0.5.0


['alexnet',
 'deeplabv3_resnet101',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'fcn_resnet101',
 'googlenet',
 'inception_v3',
 'mobilenet_v2',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext50_32x4d',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'squeezenet1_0',
 'squeezenet1_1',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vgg16_bn',
 'vgg19',
 'vgg19_bn',
 'wide_resnet101_2',
 'wide_resnet50_2']

# Imports

In [5]:
%pip install poutyne          # to install the Poutyne library
%pip install wget             # to install the wget library in order to download data
%pip install opencv-python    # to install the cv2 (opencv) library

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


In [ ]:
!pip install --upgrade torchvision>=0.6.0

In [7]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import math
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wget
import zipfile
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as tfms
#from poutyne import set_seeds, Model, ModelCheckpoint, CSVLogger, Experiment, StepLR
from torch.utils.data import DataLoader, Subset, Dataset
from torchvision.utils import make_grid

# Downloading the CelebA dataset

In [10]:
data_root = "datasets"

base_url = "https://graal.ift.ulaval.ca/public/celeba/"

file_list = [
    "img_align_celeba.zip",
    "list_attr_celeba.txt",
    "identity_CelebA.txt",
    "list_bbox_celeba.txt",
    "list_landmarks_align_celeba.txt",
    "list_eval_partition.txt",
]

# Path to folder with the dataset
dataset_folder = f"{data_root}/celeba"
os.makedirs(dataset_folder, exist_ok=True)

for file in file_list:
    url = f"{base_url}/{file}"
    if not os.path.exists(f"{dataset_folder}/{file}"):
        wget.download(url, f"{dataset_folder}/{file}")

with zipfile.ZipFile(f"{dataset_folder}/img_align_celeba.zip", "r") as ziphandler:
    ziphandler.extractall(dataset_folder)

100% [..........................................................................] 2836386 / 2836386

# Mobilenet
---
Dataset = CelebA \
Epochs = 50 \
Number of binary attributes = 40

In [12]:
import os
from shutil import copyfile
import torch
import glob
import torch.nn as nn
from PIL import Image
from torch.utils.tensorboard import SummaryWriter


# 6826/1745 = 8571

import numpy as np
#import nibabel as nib
import torch
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader
from math import ceil
from  torchvision import transforms
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split

In [13]:
if not os.path.exists(os.path.join(dataset_folder,'training_test_splits/')):
  print(dataset_folder)
  print(os.path.join(dataset_folder,'training_test_splits/'))
  os.mkdir(os.path.join(dataset_folder,'training_test_splits/'))

if not os.path.exists(os.path.join(dataset_folder,'training_test_splits/random_split/')):
  print(dataset_folder)
  print(os.path.join(dataset_folder,'training_test_splits/random_split/'))
  os.mkdir(os.path.join(dataset_folder,'training_test_splits/random_split/'))

# Complete the data path
path = os.path.join(dataset_folder,'img_align_celeba')
label_path = os.path.join(dataset_folder,'list_attr_celeba.txt')
split_path = os.path.join(dataset_folder,'training_test_splits/random_split/')

print("path=",path)
print("label_path=",label_path)
print("split_path=",split_path)

datasets/celeba
datasets/celeba\training_test_splits/
datasets/celeba
datasets/celeba\training_test_splits/random_split/
path= datasets/celeba\img_align_celeba
label_path= datasets/celeba\list_attr_celeba.txt
split_path= datasets/celeba\training_test_splits/random_split/


In [14]:
#delimeter = [' ','\t']
import csv
def load_labelfile(path):
  labels = {}
  with open(path, 'r') as csvfile:
    lines = csv.reader(csvfile, delimiter='\n')
    for i,line in enumerate(lines):
      #print(len(line))
      #print(line)
      #print(line[0].split(' ')[0])
      if(i>=2):
        #print(line[0].split()[1:])
        labels[line[0].split()[0]] =  np.array(list(map(float,line[0].split()[1:])))
  return labels

In [15]:
label_file = load_labelfile(label_path)
#print(label_file[0])
print(list(label_file.items())[2:4])

[('000003.jpg', array([-1., -1., -1., -1., -1., -1.,  1., -1., -1., -1.,  1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1.,  1.,  1., -1.,
       -1.,  1., -1., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1.,
        1.])), ('000004.jpg', array([-1., -1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.,
       -1.,  1., -1., -1., -1., -1.,  1., -1.,  1., -1.,  1.,  1., -1.,
        1.]))]


In [16]:
label_df = pd.DataFrame.from_dict(label_file,orient='index')

In [17]:
print(label_df.head())

             0    1    2    3    4    5    6    7    8    9   ...   30   31  \
000001.jpg -1.0  1.0  1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0  1.0   
000002.jpg -1.0 -1.0 -1.0  1.0 -1.0 -1.0 -1.0  1.0 -1.0 -1.0  ... -1.0  1.0   
000003.jpg -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0   
000004.jpg -1.0 -1.0  1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0   
000005.jpg -1.0  1.0  1.0 -1.0 -1.0 -1.0  1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0   

             32   33   34   35   36   37   38   39  
000001.jpg  1.0 -1.0  1.0 -1.0  1.0 -1.0 -1.0  1.0  
000002.jpg -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  1.0  
000003.jpg -1.0  1.0 -1.0 -1.0 -1.0 -1.0 -1.0  1.0  
000004.jpg  1.0 -1.0  1.0 -1.0  1.0  1.0 -1.0  1.0  
000005.jpg -1.0 -1.0 -1.0 -1.0  1.0 -1.0 -1.0  1.0  

[5 rows x 40 columns]


In [18]:
label_df_train, label_df_test = train_test_split(label_df, test_size=0.33, random_state=42)

In [19]:
print(label_df_train.head())

             0    1    2    3    4    5    6    7    8    9   ...   30   31  \
108259.jpg -1.0 -1.0  1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0  1.0   
120876.jpg -1.0 -1.0  1.0 -1.0 -1.0 -1.0  1.0  1.0 -1.0 -1.0  ... -1.0  1.0   
147440.jpg -1.0 -1.0 -1.0  1.0 -1.0 -1.0 -1.0  1.0  1.0 -1.0  ... -1.0  1.0   
140419.jpg -1.0  1.0  1.0 -1.0 -1.0 -1.0  1.0 -1.0 -1.0 -1.0  ... -1.0  1.0   
098743.jpg  1.0 -1.0  1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0   

             32   33   34   35   36   37   38   39  
108259.jpg -1.0 -1.0 -1.0 -1.0  1.0 -1.0 -1.0  1.0  
120876.jpg -1.0 -1.0 -1.0 -1.0  1.0 -1.0 -1.0  1.0  
147440.jpg  1.0 -1.0 -1.0 -1.0 -1.0 -1.0  1.0 -1.0  
140419.jpg -1.0 -1.0  1.0 -1.0  1.0 -1.0 -1.0  1.0  
098743.jpg  1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  1.0  

[5 rows x 40 columns]


In [20]:
label_df_train.to_csv(split_path+'training.txt', sep=' ', index=True,header=False,mode='a')
label_df_test.to_csv(split_path+'test.txt', sep=' ', index=True,header=False,mode='a')

In [34]:
class CustomDataset(Dataset):
  def __init__(self, folder_path, label_dict, split_file, image_size=256):
    self.label_dict = label_dict
    self.image_size = image_size
    self.split = []
    with open(split_file, 'r') as f:
      for i in f.readlines():
        self.split.append(i.strip().split()[0])
    #print("split=",self.split[0])

    self.transform = transforms.Compose([
				        transforms.Resize(self.image_size),
				        transforms.CenterCrop(self.image_size),
				        transforms.ToTensor(),
				        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
				    ])

    self.total_list = glob.glob(folder_path+'/*')
    self.total_list = [i.replace(os.sep,'/') for i in self.total_list]
    self.image_list = []

    #print("total_list=",self.total_list[0])

    for i in tqdm(self.total_list):
      #print(i)
      #print(i.split('/')[-1])
      if i.split('/')[-1] in self.split:
        #print("yes")
        self.image_list.append(i)
    #print("image_list=",self.image_list[:3])
    # Calculate len
    self.data_len = len(self.image_list)

  def __getitem__(self, index):
    single_image_path = self.image_list[index]

    im_as_im = Image.open(single_image_path)

    # Transform image to tensor, change data type
    im_as_ten = self.transform(im_as_im)
    #print(im_as_ten.shape)
    # Get label(class) of the image based on the file name
    label_idx = self.image_list[index].split('/')[-1]
    #print(label_idx)

    label = torch.Tensor(self.label_dict[label_idx])
    return (im_as_ten.cuda(), label.cuda())
    #return (im_as_ten, label)

  def __len__(self):
    #print(self.data_len)
    return self.data_len

In [22]:
def load_ckpt(path, model, optimizer):
	# ckpt = '/home/peiye/ImageEditing/scene_regressor/checkpoint/100_dict.model'

	ckpt = torch.load(path)
	model.load_state_dict(ckpt['model'])
	optimizer.load_state_dict(ckpt['optm'])
	return model, optimizer

In [35]:
if __name__ == '__main__':
	#if not os.path.exists('./checkpoint_256'):
	#	os.mkdir('./checkpoint_256')
  #	os.mkdir('./log_256')

	# Complete the data path
	#path = '/transient_scene/imageAlignedLD/'
	#label_path = '/transient_scene/annotations/annotations.tsv'
	#split_path = '/transient_scene/training_test_splits/random_split/'


	label_file = load_labelfile(label_path)

	train_data = CustomDataset(path, label_file,
								split_file=split_path+'training.txt',
								image_size=256)

	test_data = CustomDataset(path, label_file,
								split_file=split_path+'test.txt',
								image_size=256)

	trainloader = DataLoader(train_data, batch_size=32, shuffle=True)
	testloader = DataLoader(test_data, batch_size=32, shuffle=False)

100%|████████████████████████████████████████████████████████████████████████| 202599/202599 [03:18<00:00, 1019.43it/s]


In [36]:
#print(train_data.__len__())
#print(test_data.__len__())
#print(next(iter(trainloader))[0].shape)
print(train_data.__getitem__(0))

(tensor([[[-0.5451, -0.5373, -0.5373,  ..., -0.4510, -0.4353, -0.4353],
         [-0.5373, -0.5294, -0.5294,  ..., -0.4667, -0.4510, -0.4431],
         [-0.5373, -0.5294, -0.5294,  ..., -0.4745, -0.4667, -0.4667],
         ...,
         [-0.5216, -0.4980, -0.4745,  ...,  0.5137,  0.5216,  0.5137],
         [-0.4902, -0.4745, -0.4510,  ...,  0.6078,  0.6078,  0.5922],
         [-0.4667, -0.4431, -0.4196,  ...,  0.6627,  0.6627,  0.6471]],

        [[-0.6078, -0.6000, -0.6000,  ..., -0.4667, -0.4588, -0.4588],
         [-0.6000, -0.5922, -0.5922,  ..., -0.4824, -0.4824, -0.4824],
         [-0.6000, -0.5922, -0.5922,  ..., -0.5059, -0.5059, -0.5059],
         ...,
         [-0.4353, -0.4118, -0.3804,  ...,  0.5373,  0.5529,  0.5608],
         [-0.4039, -0.3882, -0.3569,  ...,  0.6235,  0.6392,  0.6471],
         [-0.3804, -0.3569, -0.3255,  ...,  0.6706,  0.6784,  0.6863]],

        [[-0.6314, -0.6235, -0.6235,  ..., -0.5059, -0.5059, -0.5137],
         [-0.6235, -0.6157, -0.6157,  ..., -

In [44]:
if __name__ == '__main__':
  files = glob.glob('./log_256/*')
  for f in files:
      os.remove(f)
  os.rmdir('./log_256')
  files = glob.glob('./checkpoint_256/*')
  for f in files:
      os.remove(f)
  os.rmdir('./checkpoint_256')
    
  if not os.path.exists('./checkpoint_256'):
      os.mkdir('./checkpoint_256')
      os.mkdir('./log_256')

  #model = torch.hub.load('pytorch/vision:v0.5.0', 'resnet50', pretrained=True)
  #print(model)
  model = torch.hub.load('pytorch/vision:v0.5.0', 'mobilenet_v2', pretrained=True)
  #print(model)
  model.classifier = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2, inplace=False),
            torch.nn.Linear(in_features=1280, out_features=40, bias=True)
        )
  #print(model)

  model = model.cuda()

  N_EPOCH = 3
  N_ITER = len(trainloader)
  optimizer = optim.Adam(model.parameters(), lr=1e-4)

  # model, optimizer = load_ckpt(path='/home/peiye/ImageEditing/scene_regressor/checkpoint/200_dict.model',
  # 							 model=model,
  # 							 optimizer=optimizer)

  writer = SummaryWriter(log_dir='./log_256'+'/')

  criterion = nn.MSELoss().cuda()
  print('EPOCH: ', N_EPOCH, 'ITER: ', N_ITER)

  pbar = tqdm(range(N_EPOCH))

  for epoch in pbar:
    it = iter(trainloader)
    for iter_ in range(N_ITER):
      data, label = next(it) # [32, 3, 256, 256]
      preds = model(data)
      optimizer.zero_grad()
      #print(label.shape)
      #print(preds.shape)
      Loss = criterion(preds, label).mean()
      Loss.backward()
      optimizer.step()

      pbar.set_description(f'Iter {iter_ + 1} Loss: {Loss:.5f}')
      if (N_ITER* epoch + iter_) % 50 == 0:
        writer.add_scalar('Train/Loss', Loss, N_ITER*epoch + iter_)

		# if epoch % 5 == 0 and epoch != 0:

    if epoch % 1 == 0 and epoch != 0:
      # Test
      with torch.no_grad():
        test_loss = []
        # aps = []

        for test_data, test_label in testloader:
          test_preds = model(test_data)
          test_loss.append(criterion(test_preds, test_label).mean().cpu().item())

					# print(test_preds.cpu().numpy().shape,test_label.cpu().numpy().shape)
					# ap = average_precision_score(test_preds.cpu().numpy(),test_label.cpu().numpy())
					# aps.append(ap)
					# break

        pbar.set_description(f'Test epoch {epoch}; Loss: {np.mean(test_loss):.5f}')
        writer.add_scalar('Test/MSE', np.mean(test_loss), epoch)
				# writer.add_scalar('Test/AP', np.mean(aps), epoch)


		# Save model
    torch.save({
              'model': model.state_dict(),
              'optm': optimizer.state_dict()},
               'checkpoint_256' + f'/{str(epoch + 1).zfill(3)}_dict.model')
  writer.close()

Using cache found in C:\Users\sanika/.cache\torch\hub\pytorch_vision_v0.5.0
  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

EPOCH:  3 ITER:  4242


Test epoch 2; Loss: 0.24551: 100%|███████████████████████████████████████████████████| 3/3 [1:29:08<00:00, 1782.81s/it]
